In [1]:
import random
import functools
import timeit
import time
import pyrankvote
from array import *
import gurobipy as gp
from gurobipy import GRB
import itertools
from stvpoll.scottish_stv import ScottishSTV
import math
import pandas as pd
import math
import copy
import numpy
from decimal import *
from decimal import Decimal
from pyrankvote import Candidate, Ballot
import inspect
from stvpoll.__init__ import ElectionRound
import bisect

In [2]:

#number of votes
#length of each ballot
most=2
x=6
global p1
global p2
global branchb
global branchm
global ILPtime
global ztime
ztime=0
global ext
ext=0
global delt
delt=0
#number of the candidates
#m = rndomBallotGen(n,l,x)

In [3]:
def tolist(listuple):
    ans=[]
    for i in ((listuple)):
        ans.append(list(i))
    return ans

In [4]:
    lstt = list(range(x))
    permt=list(itertools.permutations(lstt))
    permt.sort()
    permt=copy.deepcopy(tolist(permt))

In [5]:
global zdict
zdict={}

In [6]:
def zetta(order,k,r):
    start=time.time()
    global zdict
    zt=copy.deepcopy((tuple(order),k,r))
    if(zt in zdict):
        return zdict[zt]
    cand=[]
    for i in range(r):
        cand.append(order[i])
#    print(cand)
    
    lst = list(order)
    perm=list(itertools.permutations(lst))
    perm.sort()
 #   perm.reverse()
#    print(perm)
#    perm.sort()
    ans=[]
    ek=k
    for i in range(len(perm)):
        ind=perm[i].index(ek)
#        print(ind)
        ok=True
    #    print(i,perm[i],ind)
        for j in range(0,ind):
            if((perm[i][j] not in cand)):
            #    print(perm[j])
                ok=False
                break
        if(ok):
            ans.append(i)
    end=time.time()
    
    global ztime
    
    ztime+=(end-start)
    zdict[zt]=ans
    
        
    return ans

In [7]:
def convert(a,b):
    ans=list(copy.deepcopy(a))
 #   print("a=",a)
    for i in a:
        if i not in b:
            ans.remove(i)
#    u=list(set(b).intersection(a))
#    print(a,b,u)
#    return u
    return ans

In [8]:
def rndomBallotGen(most):
    votes  = []
    per=list(range(x))
    p=list(itertools.permutations(per))
#    print(p)
    for i in range(math.factorial(x)):
        votes.append((p[i],random.randint(0,most)))
   #     per=nextPermutation(per)
        
    return votes

In [9]:
def ILP(votes,order):
    #define model
    start=time.time()
    largeValue = 1000
    noc=len(order)
    model = gp.Model("margin")
    model.setParam("OutputFlag",0)
 #   votes=copy.deepcopy(votes[:][1])
   
    ted=math.factorial(noc)
    if(len(votes)!=math.factorial(len(order))):
        vt=[]
        for i in range(ted):
            vt.append([0,0])
        ls2 = copy.deepcopy(order)
        pr2=tolist(itertools.permutations(ls2))
        pr2.sort()
        for i in range(len(permt)):
            rw=convert(permt[i],order)
#            print(order,rw)
#            print(votes,len(perm),len(votes))
            vt[pr2.index(rw)][1]+=votes[i][1]
            vt[pr2.index(rw)][0]=permt[i]
        votes=copy.deepcopy(vt)
        lst=copy.deepcopy(ls2)
            
        
#    model.params.NonConvex = 2
    b = model.addVar(vtype=GRB.INTEGER,name='b')
    # x[i,j] = a binary variable that is 1 iff candidate j is eliminated before candidate i
    #ai,j a variable representing the number of ballots < i, j > that are added
    p = model.addVars(ted,vtype=GRB.INTEGER,name='p')
    m = model.addVars(ted,vtype=GRB.INTEGER,name='m')
    y = model.addVars(ted,vtype=GRB.INTEGER,name='y')
 #   print(votes[0])
    for i in range(ted):
        model.addConstr(y[i]<=sum(vot[1] for vot in votes))
        model.addConstr(votes[i][1]+p[i]-m[i]==y[i])
        model.addConstr(0<=y[i])
        model.addConstr(m[i]<=votes[i][1])
        model.addConstr(0<=m[i])
        model.addConstr(0<=p[i])
    model.addConstr(sum(p[i] for i in range(ted)) == sum(m[i] for i in range(ted)))
    for i in range(noc):
        for j in range(i+1,noc):
            set1=zetta(order,order[i],i)
            set2=zetta(order,order[j],i)
            p1=gp.quicksum(y[l] for l in set1)
            p2=gp.quicksum(y[l] for l in set2)
       #     print(set1,set2)
            model.addConstr(p1<=p2)
    model.addConstr(b == gp.quicksum(p[i] for i in range(ted)))
    model.setObjective(b, GRB.MINIMIZE)
    model.optimize()
    
    pa=list(p[i].X for i in range(ted))
    ya=list(y[i].X for i in range(ted))
    ma=list(m[i].X for i in range(ted))
  #  print(pa,ya,ma)
  
    #print(myvars)
    end=time.time()
    global ILPtime
    ILPtime+=(end-start)
    return model.getObjective().getValue()

In [10]:
# start=time.time()
# print(optmar(votes,0))
# end=time.time()
# print(end-start)

def deltascee(votes,order,c,e):
    start=time.time()
    global dicti
    if(((tuple(order),c)) in dicti):
        return dicti[(tuple(order),c)]
    S=copy.deepcopy(order)
    S.append(e)
    ans=0
#     lst=list(i for i in range(x))
#     perm=list(itertools.permutations(lst))
#     perm.sort()
# #    print("perm=",perm)
#     perm=copy.deepcopy(tolist(perm))
    for i in range(len(permt)):
        jayg=copy.deepcopy(convert(permt[i],S))
        if(jayg[0]==c):
            ans+=votes[i]
    global delt
    end=time.time()
    delt+=end-start
    dicti[(tuple(order),c)]=ans
    return ans

In [11]:
def deltasce(votes,order,c,e):
    start=time.time()
    global dicti
    if(((tuple(order),c)) in dicti):
        return dicti[(tuple(order),c)]
    S=copy.deepcopy(order)
    S.append(e)
    ans=0
   
#     lst=list(i for i in range(x))
#     perm=list(itertools.permutations(lst))
#     perm.sort()
# #    print("perm=",perm)
#     perm=copy.deepcopy(tolist(perm))

    for i in range(len(permt)):
        ok=True
        ja=permt[i].index(c)
        for l in range(0,ja):
            if permt[i][l] in S:
                ok=False
                break
        if ok:
            ans+=votes[i][1]
                
        
#        jayg=copy.deepcopy(convert(permt[i],S))
#        if(jayg[0]==c):
#            ans+=votes[i]
    global delt
    end=time.time()
    delt+=end-start
    dicti[(tuple(order),c)]=ans
    return ans
    

In [12]:
global fdict

In [13]:
def f(e,votes):
    ff=(e,tuple(votes))
    ans=0
    if ff in fdict:
        return fdict[ff]
    for i in range(len(permt)):
        if(permt[i][0]==e):
            ans+=votes[i][1]
    fdict[ff]=ans
    return ans
    
    

In [14]:
def l2(c,e,order,votes):
    return math.ceil((f(e,votes)-deltasce(votes,order,c,e))/2)

In [15]:
def lb2(order,votes):
    ans=0
    global dlb
    lbb=(tuple(order),tuple(votes))
    if lbb in dlb:
        return dlb[lbb]
    for c in order:
        for e in range(x):
            if(e not in order):
                ans=max(ans,l2(c,e,order,votes))
    dlb[lbb]=ans
    return ans
                

In [16]:

def expand(l,order,U,Q,votes):
    start=time.time()
    global ext
    if(len(order)==x):
#        print(votes,z,order)
        return l,Q,order
    for c in range(x):
        if c not in order:
            orpr=[c,*order]
            lpr=max(l,lb2(orpr,votes))
            if lpr<U:


                m=ILP(votes,orpr)
                global branchb
                branchb+=1
                lpr=max(lpr,m)
            if lpr<U:
 #               print("lpr=",lpr,"orpr=",orpr,"ILP=",ILP(votes,orpr))
                bisect.insort(Q,([lpr,orpr]))
#                Q.append([lpr,orpr])    
                global p2
                if(len(orpr)==x):
                    p2+=1
    end=time.time()
    ext+=end-start
    return U,Q,order

In [17]:
def blom(votes,w):
    ans=1000*1000
    pr=[]
    order=[]
#    print(perm[0])
    siz=0.0
    ted=0.0
    pr.append([0,[w]])
    while(len(pr)):
        ted+=1
        
#        pr.sort()
#        print(pr)
        it=copy.deepcopy(pr[0])
    #    print(it[0],it[1])
        pr.remove(pr[0])
        siz+=len(it[1])
        
        if(it[0]<ans):
            ans,pr,order=expand(it[0],it[1],ans,pr,votes)
    return ans,order,siz/ted


In [18]:
ted=0
maxi=10
global dicti
dicti={}
global zdict
zdict={}
global fdict
fdict={}
global dlb
dlb={}
t1=0
t2=0
p1=0
p2=0
t3=0

g3=0
fi3=0
global branchm
branchm=0
global branchb
branchb=0
while(ted<maxi):
    dicti.clear()
    zdict.clear()
    fdict.clear()
    dlb.clear()
    # t1=0
    # t2=0
    # p1=0
    # p2=0
    ted+=1
    votes=rndomBallotGen(most)
    start=time.time()
    # global branchm
    # branchm=0
    global ILPtime
  
    ztime=0
#    branchb=0
    ILPtime=0
    ext=0
    b1,b2,b3=blom(votes,0)
    fi3+=b3
    end=time.time()
    ILb=ILPtime
    t2+=end-start
#    print(p1,p2,"\n")
    # start=time.time()
    # d1,d2=margin(votes,0)
    # end=time.time()
    # t3+=end-start
   
    print("blom  ","time =",t2/maxi,"  complete signutures assessed=",p2/maxi,"  average size of assessed strings=",fi3/maxi, "uses  of ILP=",branchb/maxi," ILPtime= ",ILb,".  ztime=",ztime,"  expandtime= ",ext,"  deltascale time= ",delt)
 #   print(t2,p2,b3)
print("\n")
    

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-10
blom   time = 0.1384298324584961   complete signutures assessed= 0.1   average size of assessed strings= 0.38571428571428573 uses  of ILP= 7.6  ILPtime=  1.1798579692840576 .  ztime= 0.05185103416442871   expandtime=  1.383301019668579   deltascale time=  0.11868453025817871
blom   time = 0.27207245826721194   complete signutures assessed= 0.3   average size of assessed strings= 0.7755693581780538 uses  of ILP= 14.4  ILPtime=  1.1599462032318115 .  ztime= 0.05784130096435547   expandtime=  1.3354289531707764   deltascale time=  0.21442532539367676
blom   time = 0.6900545597076416   complete signutures assessed= 1.9   average size of assessed strings= 1.209242827565809 uses  of ILP= 24.1  ILPtime=  3.8813230991363525 .  ztime= 0.24830245971679688   expandtime=  4.176461219787598   deltascale time=  0.34006190299987793
blom   time = 0.8339697360992432   complete signutures assessed= 2.0   average s

In [19]:
## magrino example 1
##################
a = 1
b = 2
c = 3
d = 4

B = {(a,c,b,d): 40,
    (b,c,d,a): 21,
    (c,a,b,d):10,
    (c,a,d):10,
    (d,b,c,a):5}

C = (a,b,c,d)

In [22]:
w = b

In [23]:
blom(votes,w)

(1.0, [4, 5, 3, 1, 0, 2], 3.7755102040816326)